# Geolocation Backtesting Sample
## We here introduce a high-performing alternative data factor for Japan Transportation equipment tickers.
- Universe: Manufacturers for transportation Machines and equipments
    - tickers: 89
    - market cap covered: over 91 Trillion JPY
- Factor Return Performance:
    - ann. return: 13.3%
    - ann. risk: 14.4%
    - R/R: 0.92
- Factor construction in a nutshell:
    - Take Geolocation data and resample to weekly, and take year over year
    - calculate correlations (t-values) between the transformed geolocation data and company revenues for each year by walk-forward.
    - form quantile portfolios by using the transformed geolocation data with weekly rebalancing frequency
        - at each rebalancing date, use only the tickers with high correlations (t-values) of the corresponding year.

In [1]:
import sys
import numpy as np
import pandas as pd
import plotly.io
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle

for_html = False
if for_html:
    plotly.offline.init_notebook_mode()
else:
    plotly.io.renderers.default = 'iframe'


if '../..' not in sys.path:
    sys.path.append('../..')

from libs.dataset import aiq_geolocation as geo
from libs.dataset import common

In [2]:
from aiq_strategy_robot.data.data_accessor import DAL
from aiq_strategy_robot.evaluator import inv_return_stats, cumplot_return
from aiq_strategy_robot.evaluator import AltDataEvaluator
sdh = DAL()

In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
# a list of manufacturers for transportation machine & parts in Japan
sec_list = sec_lsit = ['3116', '3526', '5949', '6016', '6018', '6023', '6042', '6201', '6455', '6493', '6584', '6902', '6982', '6995', '7012', '7014', '7018', '7102', '7105', '7122', '7201', '7202', '7203', '7205', '7208', '7211', '7212', '7213', '7214', '7215', '7217', '7218', '7219', '7220', '7222', '7224', '7226', '7227', '7228', '7229', '7231', '7235', '7236', '7238', '7239', '7240', '7241', '7242', '7245', '7246', '7247', '7250', '7254', '7255', '7256', '7259', '7261', '7264', '7265', '7266', '7267', '7268', '7269', '7270', '7271', '7272', '7273', '7277', '7278', '7279', '7282', '7283', '7284', '7287', '7291', '7292', '7294', '7296', '7297', '7299', '7309', '7313', '7314', '7317', '7318', '7399', '7408', '7409', '7551']
print(len(sec_list))

89


## Data Preparation

### Load Geolocation Data

In [5]:
dfgeo = geo.read_foot_traffic_place(sec_list)
dfgeo.head()

VARIABLE               geofence  geofence_ot
TICKER DATETIME                             
3116   2014-07-01  49202.789588  5805.933658
       2014-07-02  47579.303557  5825.196556
       2014-07-03  48894.154737  5919.235621
       2014-07-04  47721.804370  5875.400565
       2014-07-05  14624.197475  4873.297697

### Get Market Data From yFinance

In [6]:
dfstock = common.read_market_data_from_yfinance(sec_list, "2014-01-01")
dfstock.head()

100%|██████████| 89/89 [00:14<00:00,  6.07it/s]

no:  7227


open        high         low       close  volume  \
TICKER DATETIME                                                             
3116   2014-01-06  978.615694  983.080860  970.429555  975.638916  692700   
       2014-01-07  974.894841  977.871620  953.313200  956.289978  782300   
       2014-01-08  954.801589  959.266756  942.150282  956.289978  999400   
       2014-01-09  957.033899  962.987453  948.847761  953.312927  883100   
       2014-01-10  949.591880  953.312851  939.173160  951.824463  847300   

                   dividends  stock splits  
TICKER DATETIME                             
3116   2014-01-06        0.0           0.0  
       2014-01-07        0.0           0.0  
       2014-01-08        0.0           0.0  
       2014-01-09        0.0           0.0  
       2014-01-10        0.0           0.0

### Load Financial Data
*Monthly resample and yoy converted data

In [7]:
sec_list = dfstock.index.get_level_values('TICKER').unique()

In [8]:
dfsales = geo.read_fundamental(sec_list)
dfsales.tail()

revenue_yoy  grossIncome_yoy
TICKER DATETIME                                
7551   2023-06-30          NaN              NaN
       2023-09-30          NaN              NaN
       2023-12-31    -0.032441        -0.041991
       2024-03-31    -0.034768        -0.092876
       2024-06-30    -0.007490        -0.074372

### Register to Handler

In [9]:
sdh_corr = DAL()

# Register the loaded datasets to sdh
data_id_sales = sdh_corr.set_raw_data(
    data_source='external',
    dfraw=dfsales
)
data_id_geo = sdh_corr.set_raw_data(
    data_source='external',
    dfraw=dfgeo
)

dfgeo_w = dfgeo.groupby('TICKER').resample('W', level='DATETIME').sum()
dfstock_w = dfstock.groupby('TICKER').resample('W', level='DATETIME').agg({'open': 'first', 'close': 'last'})

# register the transformed to sdh.
data_id_stock = sdh.set_raw_data(
    data_source='external',
    dfraw=dfstock_w
)
data_id_geo_w = sdh.set_raw_data(
    data_source='external',
    dfraw=dfgeo_w
)

sdh.set_alias({
    data_id_sales: 'sales',
    data_id_geo: 'geo',
    data_id_stock: 'stock_price',
    data_id_geo_w: 'weekly_geo',
})


In [10]:
dfgeo.index.dtypes

TICKER              object
DATETIME    datetime64[ns]
dtype: object

In [11]:
dfgeo.groupby('TICKER').resample('W', level='DATETIME').sum()

VARIABLE                geofence   geofence_ot
TICKER DATETIME                               
3116   2014-07-06  219582.097068  31862.857166
       2014-07-13  261828.830183  35760.383617
       2014-07-20  263737.603853  35645.244282
       2014-07-27  241974.862387  34757.002238
       2014-08-03  260584.144061  37372.793674
...                          ...           ...
7551   2024-06-02    1037.952169    338.973425
       2024-06-09    1069.307690    363.013144
       2024-06-16    1035.995465    360.469725
       2024-06-23     980.743412    370.894815
       2024-06-30     923.558852    348.026616

[44370 rows x 2 columns]

## Compute Correlation: Sales vs. Geolation

In [12]:
ade_corr = AltDataEvaluator(sdh_corr)

In [13]:
# check the registered data.
sdh_corr.get_raw_data(data_id_sales)

revenue_yoy  grossIncome_yoy
TICKER DATETIME                                
3116   2014-09-30          NaN              NaN
       2014-12-31          NaN              NaN
       2015-03-31          NaN              NaN
       2015-06-30          NaN              NaN
       2015-09-30     0.122563         0.283090
...                        ...              ...
7551   2023-06-30          NaN              NaN
       2023-09-30          NaN              NaN
       2023-12-31    -0.032441        -0.041991
       2024-03-31    -0.034768        -0.092876
       2024-06-30    -0.007490        -0.074372

[3520 rows x 2 columns]

In [14]:
# Transform Geolocation data to create factor values.
sdh_corr.transform.clear()
format_id_geo = sdh_corr.transform.resample(data_id=data_id_geo, fields='geofence_ot', rule='Q', func='sum').log_diff(4, names='geofence_ot_yoy').variable_ids
format_id_sales = sdh_corr.transform.raw(data_id=data_id_sales, fields=['revenue_yoy', 'grossIncome_yoy'], names=['revenue_yoy', 'grossIncome_yoy']).variable_ids
sdh_corr.transform.definition

,variable_id,data_id,data_source,source,table,field,ticker,reference_id,variable_type,method,params,process_id,return_type
variable_name,,,,,,,,,,,,,
"Unknown2_Geofence_ot_Resampled(Q|sum, origin=""start_day"")",1,2,external,Unknown,Unknown,geofence_ot,None,NaN,feature,resample,"rule=""Q"", func=""sum"", origin=""start_day""",1,None
geofence_ot_yoy,2,2,external,Unknown,Unknown,"Unknown2_Geofence_ot_Resampled(Q|sum, origin=""...",None,1.0,feature,log_diff,periods=4,2,None
revenue_yoy,3,1,external,Unknown,Unknown,revenue_yoy,None,NaN,feature,raw,,3,None
grossIncome_yoy,4,1,external,Unknown,Unknown,grossIncome_yoy,None,NaN,feature,raw,,3,None


### Identify which pairs of (ticker, year) show high correlations between Geo data and company fundamentals

In [15]:
idex = 'revenue_yoy'
features = [idex, 'grossIncome_yoy']
objective = 'geofence_ot_yoy'
corr_thresh = 1.5
high_corr_df = pd.DataFrame()
for i in tqdm(range(2015, 2024)):
    till_date = str(i)+'-12-31'
    rho = ade_corr.compu_rho(features, objective, by='TICKER',tod=till_date)
    rho = rho.T.xs('t-val', level=1).sort_values(idex, ascending=False)

    # just pick those tickers with high correlations with revenue
    high_corr = rho.loc[rho[idex] > corr_thresh, [idex]]
    
    high_corr['till_year'] = i
    high_corr = high_corr.reset_index().set_index(['TICKER', 'till_year'])
    high_corr_df = pd.concat([high_corr_df, high_corr], axis=0)
high_corr_df

100%|██████████| 9/9 [00:00<00:00,  9.14it/s]


revenue_yoy
TICKER till_year             
3116   2016          3.815836
6995   2016          1.697749
       2017          4.985821
3116   2017          3.852242
7282   2017          3.721042
...                       ...
7270   2023          2.200730
6995   2023          2.168750
7250   2023          1.904635
7201   2023          1.901114
7242   2023          1.697749

[90 rows x 1 columns]

In [16]:
high_corr_df.groupby('till_year').count()

,revenue_yoy
till_year,
2016,2
2017,6
2018,8
2019,11
2020,13
2021,16
2022,19
2023,15


## Backtesting

In [17]:
# Run data transformations for geo data
sma = 4
diff = 1
x_id = sdh.transform.sma(data_id=data_id_geo_w, fields=['geofence_ot'], periods=sma, min_periods=1).log_diff(periods=diff).variable_ids[0]

# Transform stock prices to returns as well. 
open_ret = sdh.transform.pct_change(1, data_id=data_id_stock, fields='open', names='ret').shift(-2).variable_ids[0]
_ = sdh.transform.set_return_type()
close_ret = sdh.transform.pct_change(1, data_id=data_id_stock, fields='close', names='ret').shift(-1).variable_ids[0]
_ = sdh.transform.set_return_type()

In [18]:
# Initialize AltDataEvaluator
ade = AltDataEvaluator(sdh)

In [19]:
# define the parameters for factor choice.
nq = 3
# exe_cost = 0.0005
exe_cost = 0.0
corr_start_year = 2020
corr_end_year = high_corr_df.reset_index()['till_year'].max()

In [20]:
%%time
dfqret_d_list = []
dfsigqt_d_list = [] 

# run backtest year by year and stitch the results together.
for year in range(corr_start_year, corr_end_year+1):
    # get the list of tickers with high correlations with revenue only
    year_tic_list = high_corr_df.xs(year, level='till_year').index.to_list()

    # single year backtest   
    fromd = str(year+1) + '-01-01'
    tod = str(year+1) + '-12-31'
    dfqret_d, stats, dfsigqt_d = ade.q_backtest(
        x_id,
        # close_ret,
        open_ret,
        nq=nq,
        qmax=None,
        qmin=None,
        exe_cost=exe_cost,
        plot=False,
        stats=False,
        universe = year_tic_list,
        fromd = fromd,
        tod = tod,
        # run_mlflow = False
    )
    dfqret_d_list.append(dfqret_d.loc['001'])
    dfsigqt_d_list.append(dfsigqt_d['001'])

# stack up the backtest result year by year.
all_dfqret_df = pd.concat(dfqret_d_list)
all_dfsigqt_df = pd.concat(dfsigqt_d_list)

display(inv_return_stats(all_dfqret_df))
cumplot_return(all_dfqret_df)

,cum.Ret,ann.Ret,ann.Std,R/R,Win_R,Max_DD,Calmar Ratio
#1,0.363716,0.104204,0.228629,0.455779,0.546448,-0.279028,0.373455
#2,0.666297,0.190894,0.254329,0.750578,0.590164,-0.260307,0.733341
#3,0.862617,0.247139,0.248965,0.992666,0.606557,-0.266811,0.926269
#3-#1,0.498900,0.142935,0.144589,0.988556,0.573770,-0.220933,0.646959


CPU times: user 1.19 s, sys: 1.02 s, total: 2.21 s
Wall time: 3.29 s


## Appendix: Try different choices of moving average period and ratio period by running backtests per pairs

In [21]:
### Create an input parameter space.
# list of periods for taking ratios.
log_diff_list = [('log_diff', dict(periods=1)),
                ('log_diff', dict(periods=4)),
                ('log_diff', dict(periods=13)),
                ('log_diff', dict(periods=26)),
                ('log_diff', dict(periods=52)),
                 ]

# list of periods for taking moving averages. 
sma_list = []
for i in [1, 4, 13]:
    t_list = [('sma', dict(periods=int(f'{i}'), min_periods=1)),]
    sma_list = sma_list + t_list

# put into a dictionary the combination of the two lists
pipe_dic = {}
for log_diff in log_diff_list:
    for sma in sma_list:
        name = f'{sma[0]}_{sma[1]["periods"]}_{log_diff[0]}_{log_diff[1]["periods"]}'
        pipe = [sma, log_diff]
        pipe_dic[name] = pipe

In [22]:
feats_id_dic = {}

# run data transformation per scenario defined in pipe_dic.
for pipe_name in tqdm(pipe_dic.keys()):
    sma = pipe_dic[pipe_name][0]
    log_diff = pipe_dic[pipe_name][1]
    sma_periods = sma[1]['periods']
    sma_min_periods = sma[1]['min_periods']
    log_diff_periods = log_diff[1]['periods']

    # run the transformations per scenario
    feats_id_dic[pipe_name] = sdh.transform.sma(data_id=data_id_geo_w, fields=['geofence_ot'], periods=sma_periods, min_periods=sma_min_periods).log_diff(periods=log_diff_periods).variable_ids[0]

100%|██████████| 15/15 [00:00<00:00, 109.00it/s]


In [23]:
# check if transformed time series data has been generated.
display(sdh.transform_definition.head())

,variable_id,data_id,data_source,source,table,field,ticker,reference_id,variable_type,method,params,process_id,return_type
variable_name,,,,,,,,,,,,,
Unknown_Open,1,1,external,Unknown,Unknown,open,None,NaN,reference,raw,,1,None
Unknown_Close,2,1,external,Unknown,Unknown,close,None,NaN,reference,raw,,1,None
Unknown2_Geofence,3,2,external,Unknown,Unknown,geofence,None,NaN,reference,raw,,2,None
Unknown2_Geofence_ot,4,2,external,Unknown,Unknown,geofence_ot,None,NaN,reference,raw,,2,None
"WeeklyGeo_Geofence_ot_SMA(004, min_periods=1)",5,2,external,Unknown,Unknown,geofence_ot,None,NaN,reference,sma,"periods=4, min_periods=1",3,None


In [24]:
# define the parameters for factor choice.
nq = 3
# exe_cost = 0.0005
exe_cost = 0.0
corr_start_year = 2020
corr_end_year = high_corr_df.reset_index()['till_year'].max()

In [25]:
%%time
return_dic = {}
tic_num_dic = {}
result_list = []

# looping over scenarios.
for feat_id in tqdm(feats_id_dic.keys()):
    dfqret_d_list = []
    dfsigqt_d_list = []
    # run this per year.
    for year in range(corr_start_year, corr_end_year+1):
        year_tic_list = high_corr_df.xs(year, level='till_year').index.to_list()
        # defined backtest period to an year
        fromd = str(year+1) + '-01-01'
        tod = str(year+1) + '-12-31'
        dfqret_d, stats, dfsigqt_d = ade.q_backtest(
            feats_id_dic[feat_id], # Specify using only integer types.
            # close_ret,
            open_ret,
            nq=nq,
            qmax=None,
            qmin=None,
            exe_cost=exe_cost,
            plot=False,
            stats=False,
            universe = year_tic_list,
            fromd = fromd,
            tod = tod,
            # run_mlflow = False
        )
        dfqret_d_list.append(dfqret_d.loc['001'])
        dfsigqt_d_list.append(dfsigqt_d['001'])
    
    # consolidate the backtest result per year.
    all_dfqret_df = pd.concat(dfqret_d_list)
    all_dfsigqt_df = pd.concat(dfsigqt_d_list)
    
    # backtest stats are calculated.
    all_stats = inv_return_stats(all_dfqret_df)
    all_stats = all_stats.loc[f'#{nq}-#1',:].to_frame().T
    all_stats = all_stats.reset_index()
    all_stats = all_stats.drop(columns='index')
    all_stats['features'] = feat_id
    all_stats = all_stats.set_index('features')
    result_list.append(all_stats)

    # a return stats is stored.
    return_dic[feat_id] = all_dfqret_df

    # a dataframe for per period signals is also stored. 
    tic_num_dic[feat_id] = all_dfsigqt_df

# stack up the backtest stats over all the scenarios.
whole_result = pd.concat(result_list)
whole_result = whole_result.sort_values('R/R', ascending=False)

100%|██████████| 15/15 [00:49<00:00,  3.29s/it]

CPU times: user 15.5 s, sys: 17.8 s, total: 33.2 s
Wall time: 49.3 s


In [26]:
# Dipslay the whole result below
display(whole_result)

,cum.Ret,ann.Ret,ann.Std,R/R,Win_R,Max_DD,Calmar Ratio
features,,,,,,,
sma_4_log_diff_1,0.498900,0.142935,0.144589,0.988556,0.573770,-0.220933,0.646959
sma_1_log_diff_4,0.463643,0.132833,0.144843,0.917085,0.568306,-0.228599,0.581076
sma_1_log_diff_1,0.374225,0.107215,0.136462,0.785678,0.519126,-0.254953,0.420529
sma_4_log_diff_26,0.062370,0.017869,0.144052,0.124045,0.508197,-0.270438,0.066074
sma_4_log_diff_4,0.009075,0.002600,0.136509,0.019046,0.486339,-0.282244,0.009212
sma_1_log_diff_26,0.008528,0.002443,0.134326,0.018190,0.448087,-0.289043,0.008453
sma_4_log_diff_13,-0.012571,-0.003601,0.125549,-0.028686,0.469945,-0.203975,-0.017656
sma_13_log_diff_4,-0.048286,-0.013834,0.125473,-0.110253,0.469945,-0.209754,-0.065952
sma_13_log_diff_1,-0.137370,-0.039356,0.123286,-0.319229,0.486339,-0.252533,-0.155846


In [27]:
key = "sma_4_log_diff_1"
print(key)
pd.options.display.precision = 3
display(inv_return_stats(return_dic[key]))
cumplot_return(return_dic[key])

sma_4_log_diff_1


,cum.Ret,ann.Ret,ann.Std,R/R,Win_R,Max_DD,Calmar Ratio
#1,0.364,0.104,0.229,0.456,0.546,-0.279,0.373
#2,0.666,0.191,0.254,0.751,0.590,-0.260,0.733
#3,0.863,0.247,0.249,0.993,0.607,-0.267,0.926
#3-#1,0.499,0.143,0.145,0.989,0.574,-0.221,0.647
